In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import Select
from time import sleep
from urllib.parse import quote
import threading
from queue import Queue
import pandas as pd
import os
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
# Função para pegar os links do botão detalhar
def get_href(elements):
    hrefs = []
    for elemento in elements:
        href = elemento.get_attribute('href')
        hrefs.append(href)

    return hrefs

# Função para acessar o portal com os parametros desejados
def consultar(navegador, beneficio, start_date, end_date, municipio = 'Tanguá'):
    municipio = quote(municipio)
    navegador.get(f'https://portaldatransparencia.gov.br/beneficios/{beneficio}?de={start_date}&ate={end_date}&nomeMunicipio={municipio}&ordenarPor=nis&direcao=asc')

# Função para verificar se o número da página é maior que o valor anteriormente armazenado
def page_number_greater(driver, previous_page, id = 'lista_info'):
    page_text = driver.find_element(By.ID, id).text
    current_page = int(page_text.split()[1])
    return current_page > previous_page

In [ ]:
beneficio = 'bolsa-familia'
start_date = '01/01/2020'
end_date = '31/12/2020'
municipio = 'Tanguá'

# Iniciando drive do Chorme
driver = webdriver.Chrome()

# inicia a pagiana e espera 5 segundos para os dados serem carregados
consultar(driver, beneficio, start_date, end_date, municipio)
sleep(5)

# Clica no botão de ver paginação completa
driver.find_element(By.CLASS_NAME, 'botao__gera_paginacao_completa').click()
sleep(5)

# Seleciona a opção de 50 individuos por páginas
elemento_dropdown  = driver.find_element(By.NAME, 'lista_length')
Select(elemento_dropdown).select_by_value('50')
sleep(5)

# Verifica quantas paginas no total
quantPaginas = int(driver.find_element(By.ID, 'lista_info').text.split()[-1])

# Crie uma lista para armazenar os valores de atributo href
hrefs = []
previous_page = 1
for _ in range(quantPaginas):
    # Busca os lnkes do detalhar
    elements = driver.find_elements(By.CLASS_NAME, 'linkRendered')

    # Tenta obeter o link, caso a requisição demore para responder, espere mais 5 segundos
    try:
        link = get_href(elements)
    except:
        sleep(5)
        elements = driver.find_elements(By.CLASS_NAME, 'linkRendered')
        link = get_href(elements)


    # Adiciona na lista de links
    hrefs.extend(link)
    if previous_page != quantPaginas:
        # Clica no botão proximo  
        driver.find_element(By.ID, 'lista_next').click()

        # Aguardar a mudança de página
        WebDriverWait(driver, 10).until(lambda driver: page_number_greater(driver, previous_page))
        previous_page += 1

driver.quit()

## Transformando links em csv

In [39]:
df = pd.DataFrame(hrefs, columns=['Detalhes_link'])

In [40]:
ano = start_date.split('/')[-1]

df.to_csv(f'{start_date}.csv')

In [105]:
df = pd.read_csv('2017.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df.to_csv('bolsa-familia-2017.csv')

In [176]:
# Lista para armazenar os dados dos arquivos CSV
dados_csv = []

# Percorra todos os arquivos na pasta
for nome_arquivo in os.listdir('Dados/'):
    if nome_arquivo.endswith('.csv'):
        # Construa o caminho completo para cada arquivo CSV
        caminho_arquivo = os.path.join('Dados/', nome_arquivo)
        
        # Leia o arquivo CSV e adicione os dados à lista
        df = pd.read_csv(caminho_arquivo)
        dados_csv.append(df)

# Concatene todos os DataFrames em um único DataFrame
dados_completos = pd.concat(dados_csv)

In [177]:
dados_completos.drop('Unnamed: 0', axis=1, inplace=True)
dados_completos.drop_duplicates(inplace=True)
dados_completos.reset_index(drop=True, inplace=True)
dados_completos.to_csv('Dados/bolsa-familia.csv', index=False)

# Teste de extrair quantidade da tabela

In [20]:
driver = webdriver.Chrome()
driver.get('https://portaldatransparencia.gov.br/beneficios/bolsa-familia/301738618?ordenarPor=mesReferencia&direcao=desc')
sleep(3)
user_data = driver.find_element(By.CLASS_NAME, 'dados-tabelados').find_element(By.CLASS_NAME, 'row').find_elements(By.TAG_NAME, 'div')[-1]
name = user_data.get_attribute("innerHTML").split()[-1]


In [37]:
df = pd.read_csv('Dados/bolsa-familia.csv')
link_list = df.values.tolist()

In [40]:
driver = webdriver.Chrome()
driver.maximize_window()

dados = []

# Percorre cada link
for link in link_list:
    try:
        driver.get(link[0])
    except:
        sleep(10)
        driver.get(link[0])
    sleep(3)

    # Localize o NIS
    user_data = driver.find_element(By.CLASS_NAME, 'dados-tabelados').find_element(By.CLASS_NAME, 'row').find_elements(By.TAG_NAME, 'div')[-1]
    nis = user_data.get_attribute("innerHTML").split()[-1]

    # Localize a tabela geral
    tabela = driver.find_element(By.ID, 'tabelaDetalheValoresRecebidos_wrapper')
    tabela.find_element(By.CLASS_NAME, 'botao__gera_paginacao_completa').click()
    sleep(5)

    # Seleciona a opção de 50 individuos por páginas
    elemento_dropdown  = driver.find_element(By.NAME, 'tabelaDetalheValoresRecebidos_length')
    Select(elemento_dropdown).select_by_value('50')
    sleep(5)

    # Verifica quantas paginas no total
    quantPaginas = int(driver.find_element(By.ID, 'tabelaDetalheValoresRecebidos_info').text.split()[-1])
    
    # Pega a tabela principal
    tabela = driver.find_element(By.ID, 'tabelaDetalheValoresRecebidos')
    
    previous_page = 1
    for _ in range(quantPaginas):

        # Gambiarra para tirar o mouse de tootip em cima dos dados
        driver.find_element(By.ID, 'tabelaDetalheValoresRecebidos_info').click()
        sleep(0.1)

        # Obtenha todas as linhas da tabela
        linhas = tabela.find_elements(By.TAG_NAME, 'tr')[1:]

        # Itere sobre as linhas
        for linha in linhas:
            # Obtenha todas as células da linha
            celulas = linha.find_elements(By.TAG_NAME, 'td')
            
            # Extraia os dados das células
            dados_linha = [celula.text for celula in celulas]

            # Adicionando o nis
            dados_linha.append(nis)

            # Armazene os dados
            dados.append(dados_linha)

        if previous_page != quantPaginas:
            # Clica no botão proximo  
            driver.find_element(By.ID, 'tabelaDetalheValoresRecebidos_next').click()

            # Aguardar a mudança de página
            WebDriverWait(driver, 10).until(lambda driver: page_number_greater(driver, previous_page, 'tabelaDetalheValoresRecebidos_info'))
            previous_page += 1
        
# Feche o navegador
driver.quit()

In [34]:
header = ['Mes Folha', 'Mes Referência', 'UF', 'Municipio', 'Quantidade Dependentes', 'Valor', 'NIS']
pd.DataFrame(dados, columns=header).to_csv('detalhes-bolsa-familia.csv')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".dados-tabelados"}
  (Session info: chrome=114.0.5735.90)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00B8A813+48355]
	(No symbol) [0x00B1C4B1]
	(No symbol) [0x00A25358]
	(No symbol) [0x00A509A5]
	(No symbol) [0x00A50B3B]
	(No symbol) [0x00A7E232]
	(No symbol) [0x00A6A784]
	(No symbol) [0x00A7C922]
	(No symbol) [0x00A6A536]
	(No symbol) [0x00A482DC]
	(No symbol) [0x00A493DD]
	GetHandleVerifier [0x00DEAABD+2539405]
	GetHandleVerifier [0x00E2A78F+2800735]
	GetHandleVerifier [0x00E2456C+2775612]
	GetHandleVerifier [0x00C151E0+616112]
	(No symbol) [0x00B25F8C]
	(No symbol) [0x00B22328]
	(No symbol) [0x00B2240B]
	(No symbol) [0x00B14FF7]
	BaseThreadInitThunk [0x767A7D59+25]
	RtlInitializeExceptionChain [0x77A7B74B+107]
	RtlClearBits [0x77A7B6CF+191]


In [12]:
header = ['Mes Folha', 'Mes Referência', 'UF', 'Municipio', 'Quantidade Dependentes', 'Valor', 'NIS']
pd.DataFrame(data, columns=header)

,Mes Folha,Mes Referência,UF,Municipio,Quantidade Dependentes,Valor,NIS
0,10/2021,10/2021,RJ,TANGUÁ,2,"147,00",2.037.260.299-6
1,09/2021,09/2021,RJ,TANGUÁ,2,"147,00",2.037.260.299-6
2,08/2021,08/2021,RJ,TANGUÁ,2,"147,00",2.037.260.299-6
3,07/2021,07/2021,RJ,TANGUÁ,1,"147,00",2.037.260.299-6
4,06/2021,06/2021,RJ,TANGUÁ,1,"147,00",2.037.260.299-6
...,...,...,...,...,...,...,...
295,04/2016,04/2016,RJ,TANGUÁ,1,"232,00",2.124.893.002-0
296,03/2016,03/2016,RJ,TANGUÁ,1,"232,00",2.124.893.002-0
297,02/2016,02/2016,RJ,TANGUÁ,1,"232,00",2.124.893.002-0
298,01/2016,01/2016,RJ,TANGUÁ,1,"232,00",2.124.893.002-0


# Testando Threads

In [ ]:
def getQuantPaginas():
    driver = webdriver.Chrome()
    consultar(driver,'bolsa-familia', '01/01/2016', '31/12/2016', 'Tanguá')
    # inicia a pagian e espera 10 segundos para os dados carregarem
    sleep(5)

    # Clica no botão de ver paginação completa
    # driver.find_element(By.XPATH, '//*[@id="lista_wrapper"]/div/div[3]/div[3]/button').click()
    driver.find_element(By.CLASS_NAME, 'botao__gera_paginacao_completa').click()



    # Clica no select para aumentar o numero de dados na pagina
    elemento_dropdown  = driver.find_element(By.XPATH, '//*[@id="lista_length"]/label/select')
    Select(elemento_dropdown).select_by_value('50')

    sleep(5)
    # Verifica quantas paginas possuem
    quantPaginas = driver.find_element(By.XPATH, '//*[@id="lista_info"]').text.split()[-1]
    driver.quit()
    return int(quantPaginas)

In [ ]:
# Função que será executada em cada thread
def coletar_dados(start_page, end_page, beneficio, start_date, end_date, municipio='Tanguá'):
    driver = webdriver.Chrome()
    driver.execute_script("window.open('about:blank', 'new_tab');")
    driver.switch_to.window(driver.window_handles[-1])
    consultar(driver, beneficio, start_date, end_date, municipio)
    
    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'botao__gera_paginacao_completa'))
        )
    finally:
        driver.find_element(By.CLASS_NAME, 'botao__gera_paginacao_completa').click()

    sleep(5)
    elemento_dropdown = driver.find_element(By.XPATH, '//*[@id="lista_length"]/label/select')
    Select(elemento_dropdown).select_by_value('50')

    sleep(6)
    driver.find_element(By.ID, 'paginas-selecao-1-lista').send_keys(start_page)
    driver.find_element(By.ID, 'botao-ir-para-a-pagina-lista').click()
    sleep(7)
    hrefs = []
    for _ in range(start_page, end_page + 1):
        elements = driver.find_elements(By.CLASS_NAME, 'linkRendered')
        hrefs.extend(get_href(elements))
        driver.find_element(By.ID, 'lista_next').click()
        sleep(10)

    driver.quit()

    if results_queue:
        results_queue.put(hrefs)


# Execução das threads
beneficio = 'bolsa-familia'
start_date = '01/01/2018'
end_date = '31/12/2018'
municipio = 'Tanguá'

num_threads = 6
quantPaginas = getQuantPaginas()
pages_per_thread = quantPaginas // num_threads

threads = []
results_queue = Queue()

for i in range(num_threads):
    start_page = i * pages_per_thread + 1
    end_page = start_page + pages_per_thread - 1 if i < num_threads - 1 else quantPaginas
    thread = threading.Thread(target=coletar_dados, args=(start_page, end_page, beneficio, start_date, end_date, municipio))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()

# Recupera os resultados das threads
hrefs = []
while not results_queue.empty():
    hrefs.extend(results_queue.get())
